In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

sns.set_style("whitegrid")

Years

In [2]:
#Years we concerned with 
start_year=1990
end_year=2017
year_range = range(start_year, end_year+1) 

Some Functions

In [3]:
###
##

# Data Reads

## Salmon Survivals

In [4]:
#Read ZooPlanktonPerryData file
##obtain SalmonSurvivals_data
            #Take years less than or equal to 2017
                            #last row (year 2018) has only NaN


file_path = "Data/ZooPlanktonPerryData.xlsx"
selected_sheet = "5. Salmon marine survivals"  
df = pd.read_excel(file_path, sheet_name=selected_sheet)
SalmonSurvivals_data = df[df["Ocean Entry Year"]<=2017]


In [5]:
#Rename the columns 
column_mapping = {
    'Ocean Entry Year': 'year',
    'Cowichan Chinook': 'Cowichan_Chinook',
    'Harrison Chinook': 'Harrison_Chinook',
    'Puntledge Chinook': 'Puntledge_Chinook',
    'Big Qualicum Coho': 'Big_Qualicum_Coho'
}

salmonSurvivals = SalmonSurvivals_data.rename(columns=column_mapping)

In [6]:
# List of salmon types 
salmonTypes_List = ['Cowichan_Chinook', 'Harrison_Chinook', 'Puntledge_Chinook', 'Big_Qualicum_Coho']


## Feeding

### Total Zooplankton Biomasses

In [7]:
#Read ZooPlanktonPerryData file

##obtain  plankBiomass_data   
###take the years less than 2017 
###we drop 
file_path = "Data/ZooPlanktonPerryData.xlsx"
selected_sheet = "1. Zooplankton Biomass data"  
df = pd.read_excel(file_path, sheet_name=selected_sheet)
df= df[df["yr"]<=2017]
plankBiomass_data=df.drop(columns=['key', 'survey', 'region', 'event', 'net', 'station', 'lon', 'lat', 'mon', 'day', 'time',  'twilight', 'net.type', 'diam.m', 'mesh.um', 'startz.m', 'endz.m','botz.m',
       'volfilt.m3'])

##  BiomassDefs      (if info on column names is needed)
selected_sheet = "2. Biomass data definitions"  
BiomassDefs = pd.read_excel(file_path, sheet_name=selected_sheet)


In [8]:
## Create a new DataFrame which contains years and corresponding total biomasses mean
#Calculate average Total Biomass for each year inside the common year range
#drop duplicates
#reset index

totalBiomass_av= pd.DataFrame()

totalBiomass_av['year']= plankBiomass_data['yr']
totalBiomass_av['Av_totalBiomass']= plankBiomass_data.groupby('yr')['Total.Biomass'].transform('mean')
totalBiomass_av.drop_duplicates(inplace=True)
totalBiomass_av.reset_index(inplace=True, drop=True)


In [9]:
# Create a new DataFrame for averaged values
#Group by 'yr' and calculate the annual mean of Zooplanktons
#drop duplicates
#rename the columns for convenienve
#reset_index

plankBiomassMean = pd.DataFrame()


for column in plankBiomass_data.columns:
    if column in ['yr', 'region']:
        plankBiomassMean[column] = plankBiomass_data[column]
    else:
        name = 'Av_' + column
        plankBiomassMean[name] = plankBiomass_data.groupby('yr')[column].transform('mean')

plankBiomassMean.drop_duplicates(subset=['yr'], inplace=True)
plankBiomassMean.rename(columns={'yr': 'year', 'Av_Total.Biomass': 'Av_totalBiomass'},  inplace=True)
plankBiomassMean.reset_index(inplace=True)


### Cleaned data

In [10]:
display(plankBiomassMean.head(3))
display(totalBiomass_av.head(3))
#Note: data of totalBiomass_av appears in plankBiomassMean

,index,year,Av_AmphiGam,Av_AmphiHyp,Av_BenthicLarv,Av_CalCops.larg,Av_CalCops.med,Av_CalCops.smal,Av_Cephalopoda,Av_Chaetognatha,...,Av_Natantia,Av_NonCalCops,Av_Ostracoda,Av_Other,Av_PolychaetPelagic,Av_Pteropods,Av_Repantia,Av_Scyphozoa,Av_Siphonophorae,Av_totalBiomass
0,0,1996,3.099832,1.324307,0.274767,13.213023,9.285405,0.929795,0.015525,0.358527,...,0.718163,0.322313,2.674950,0.083352,1.615180,0.324095,0.081970,0.0,0.531088,37.388238
1,4,1997,8.415328,1.458525,0.033581,24.451942,9.381715,0.810660,0.022965,1.561735,...,0.606682,0.358207,3.035919,0.041017,0.437920,0.482410,4.063566,0.0,0.431464,71.212723
2,17,1998,8.007227,3.214592,0.572927,12.494889,11.405672,1.033331,0.053075,0.980033,...,0.637259,0.325437,3.960144,0.004294,0.897803,2.674066,0.688282,0.0,0.202136,105.522363


,year,Av_totalBiomass
0,1996,37.388238
1,1997,71.212723
2,1998,105.522363


## Water Qualities


### Sea Level

In [11]:
## Read the Annual Mean Sea Levels
        #there are data of four ports
#Take the years between start_year and end_year
#Take the positive values (non existent data entered as -99999
#drop the columns 2 and 3 to keep year and sea level measurement
#rename the column that contains Sea Level measurement


file_path="Data/SeaLevel_Point_Atkinson193.txt"
df=pd.read_csv(file_path, header=None,delimiter=';')
df= df[(df[0] >= start_year) & (df[0] <= end_year) & (df[1]>=0)].drop(columns=[2,3])
seaLevelPointA_clean =df.rename(columns={0: 'year', 1: 'SeaLevel_Point_Atkinson'})
#missing year 1997

file_path="Data/SeaLevel_Port_Angeles2127.txt"
df =pd.read_csv(file_path, header=None,delimiter=';')
df= df[(df[0] >= start_year) & (df[0] <= end_year) & (df[1]>=0)].drop(columns=[2,3])
seaLevelPortA_clean= df.rename(columns={0: 'year', 1: 'SeaLevel_Port_Angeles'})
#no missing year

file_path="Data/SeaLevel_Campbell_River1323.txt"
df=pd.read_csv(file_path, header=None,delimiter=';')
df= df[(df[0] >= start_year) & (df[0] <= end_year) & (df[1]>=0)].drop(columns=[2,3])
seaLevelCampbellR_clean= df.rename(columns={0: 'year', 1: 'SeaLevel_Campbell_River'})
#missing year 1995 & 1996

file_path="Data/SeaLevel_Cherry_annualMean.txt"
df= pd.read_csv(file_path, header=None,delimiter=';')
df= df[(df[0] >= start_year) & (df[0] <= end_year) & (df[1]>=0)].drop(columns=[2,3])
seaLevelCherryP_clean= df.rename(columns={0: 'year', 1: 'SeaLevel_Cherry'})
#missing year 1994

In [12]:
#Merge the sea Levels
#Calculate their average 
    #missing data is ignored 
        #note: if a year is missing it's only missing at most one of the ports

merged_df= pd.merge(seaLevelPortA_clean, seaLevelPointA_clean, on='year', how='outer')
merged_df = pd.merge(merged_df, seaLevelCampbellR_clean, on='year', how='outer')
seaLevels = pd.merge(merged_df, seaLevelCherryP_clean, on='year', how='outer')

seaLevels['av_SeaLevels']= seaLevels.iloc[:,1:].mean(axis=1, skipna=True)


### Salinity

In [13]:
####

### Cleaned data

Cleaned data frame for this section: seaLevels

In [14]:
seaLevels.head(3)

,year,SeaLevel_Port_Angeles,SeaLevel_Point_Atkinson,SeaLevel_Campbell_River,SeaLevel_Cherry,av_SeaLevels
0,1990,6940,7048.0,7036.0,7002.0,7006.5
1,1991,6923,7047.0,7031.0,7007.0,7002.0
2,1992,7007,7109.0,7083.0,7069.0,7067.0


# Predators

Predators in nature (From Roy's EDA)

In [15]:
#Read Seal and Whale data

Seal_file = pd.read_excel('Pred_env_data/harbourseals_1970to2020.xlsx','harbourseals_1970to2020')
Whale_file = pd.read_excel('Pred_env_data/Misc_data/killerwhales_1970to2020.xlsx','killerwhales_1970to2020')

Seal_data = Seal_file.loc[(Seal_file['year'] >= start_year) & (Seal_file['year'] <= end_year)]
Whale_data = Whale_file.loc[(Whale_file['Year'] >= start_year) & (Whale_file['Year'] <= end_year)].rename(columns={'Year': 'year'})


Recreational catch (From Roy's EDA)

In [16]:
df=pd.read_excel('Pred_env_Data/Recreational reported fishery catch 1953-2012.xlsx')
recreationalCatch_data=df[(df["YEAR"]>=start_year) & (df["YEAR"]<=end_year)]

In [17]:
# Filter the DataFrame for CHINOOK and COHO SALMON
# Sum the total pieces in a given year
#Create a data frame for the yearly sum of CHINOOK and COHO SALMON pieces

chinook_data = recreationalCatch_data[recreationalCatch_data['SPECIES_DESC'] == 'CHINOOK SALMON'].rename(columns={'YEAR': 'year'})
coho_data = recreationalCatch_data[recreationalCatch_data['SPECIES_DESC'] == 'COHO SALMON'].rename(columns={'YEAR': 'year'})


chinook_sum_year = chinook_data.groupby('year')['PIECES'].sum().reset_index(name="rc_Chinook")
coho_sum_year = coho_data.groupby('year')['PIECES'].sum().reset_index(name="rc_Coho")

rec_catch= pd.merge(chinook_sum_year, coho_sum_year, on='year', how='outer')

In [18]:
#Combine Seal, Whale and Sum of Rec catch into a new data frame

merged_df= pd.merge(rec_catch, Seal_data.drop(columns=['nonSogNt', 'bcNt']), on='year', how='outer')
merged_df = pd.merge(merged_df, Whale_data.drop(columns=['SRKW', 'srkwSource', 'nrkwSource']), on='year', how='outer')

predators= merged_df.rename(columns={'sogNt': 'Seal_sogNt', 'NRKW': 'Whale_NRKW'})

### Cleaned data

Cleaned data frame for this section: predators 

In [19]:
predators.head(3)

,year,rc_Chinook,rc_Coho,Seal_sogNt,Whale_NRKW
0,1990,214618.0,697063.0,27124,194.0
1,1991,207730.0,234207.0,30358,198.0
2,1992,221024.0,721331.0,33604,204.0


# Lasso & Ridge 

## Combine data

In [20]:
#Numbers corresponds to the place of the salmon types in the  salmonTypes_List
    #for Cowichan Chinook
cc=0
        #for Harrison Chinook
hc=1
        #for Puntledge Chinook
pc=2
        #for Big Qualicum Coho
bqc=3

In [21]:
# Combine the features we want to consider
##Drop NAN 
###The below combines [salmonSurvivals, predators, seaLevels, totalBiomass_av]

merged_df= pd.merge(salmonSurvivals, predators, on='year', how='outer')
merged_df = pd.merge(merged_df, seaLevels[['year', 'av_SeaLevels']], on='year', how='outer')
merged_df = pd.merge (merged_df, totalBiomass_av, on='year', how='outer')

salmonSurvivals__predators__SeaLevels__totalBiomass_av = merged_df.dropna().reset_index(drop=True)


In [22]:
#Selected features data frame
##New data frame is created in order to use other combination/merged of features

salmonSelectedFeatures=salmonSurvivals__predators__SeaLevels__totalBiomass_av.drop(['year'], axis=1)

## Lasso for Cowichan Chinook

In [23]:
#For different Salmon type adapt the type_number
    ##Different Salmon type might need different alphas

type_number=cc
type_name=salmonTypes_List[type_number]

In [24]:
#Drop the other Salmon Types from the data

salmon= salmon_cc=salmonSelectedFeatures.drop(columns=[salmonTypes_List[i] for i in range(0, 4) if i != type_number])


In [25]:
scaler = StandardScaler()

In [26]:
X=salmon.drop([type_name], axis=1)
scaled_data = scaler.fit_transform(X)
target = salmon[type_name]

In [27]:
#Several Lasso fittings for different alpha values

alphas = [0.000001, 0.00001, 0.0001, 0.00025, 0.0005, 0.00075, 0.001, 0.0025, 0.005, 0.01]

coefs = np.zeros((len(alphas), len(X.columns)))

for i,alpha in enumerate(alphas):
    lasso = Lasso(alpha = alpha, max_iter = 1000000)
    lasso.fit(scaled_data, target)
    
    coefs[i, :] = lasso.coef_
    
lassoCoefs = pd.DataFrame(coefs, 
                index=["alpha = " + str(alpha) for alpha in alphas],
                columns=[col for col in X.columns])

##Display the coefficients
###and display the features that produces non-zero coefficient
print('Lasso Coefficients for', type_name)
display(lassoCoefs)
print("The non-zero features are:")
for alpha in alphas:
    alpha_selected=alpha
    selected_row = lassoCoefs.loc["alpha = " + str(alpha_selected)]
    lasso_model_alpha =selected_row[selected_row != 0].index.tolist()
    print('for', alpha_selected, ': ', lasso_model_alpha, 'with length ', len(lasso_model_alpha))

Lasso Coefficients for Cowichan_Chinook


,rc_Chinook,rc_Coho,Seal_sogNt,Whale_NRKW,av_SeaLevels,Av_totalBiomass
alpha = 1e-06,0.002520,-0.000757,-4.565385e-07,0.001330,0.002239,0.002879
alpha = 1e-05,0.002486,-0.000723,0.000000e+00,0.001325,0.002229,0.002873
alpha = 0.0001,0.002157,-0.000392,0.000000e+00,0.001283,0.002133,0.002812
alpha = 0.00025,0.001705,-0.000000,0.000000e+00,0.001243,0.001957,0.002663
alpha = 0.0005,0.001346,-0.000000,0.000000e+00,0.001295,0.001603,0.002226
alpha = 0.00075,0.000987,0.000000,0.000000e+00,0.001348,0.001248,0.001789
alpha = 0.001,0.000628,0.000000,0.000000e+00,0.001400,0.000893,0.001352
alpha = 0.0025,0.000000,0.000000,0.000000e+00,0.000597,0.000000,0.000000
alpha = 0.005,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
alpha = 0.01,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000


The non-zero features are:
for 1e-06 :  ['rc_Chinook', 'rc_Coho', 'Seal_sogNt', 'Whale_NRKW', 'av_SeaLevels', 'Av_totalBiomass'] with length  6
for 1e-05 :  ['rc_Chinook', 'rc_Coho', 'Whale_NRKW', 'av_SeaLevels', 'Av_totalBiomass'] with length  5
for 0.0001 :  ['rc_Chinook', 'rc_Coho', 'Whale_NRKW', 'av_SeaLevels', 'Av_totalBiomass'] with length  5
for 0.00025 :  ['rc_Chinook', 'Whale_NRKW', 'av_SeaLevels', 'Av_totalBiomass'] with length  4
for 0.0005 :  ['rc_Chinook', 'Whale_NRKW', 'av_SeaLevels', 'Av_totalBiomass'] with length  4
for 0.00075 :  ['rc_Chinook', 'Whale_NRKW', 'av_SeaLevels', 'Av_totalBiomass'] with length  4
for 0.001 :  ['rc_Chinook', 'Whale_NRKW', 'av_SeaLevels', 'Av_totalBiomass'] with length  4
for 0.0025 :  ['Whale_NRKW'] with length  1
for 0.005 :  [] with length  0
for 0.01 :  [] with length  0


### Conclusion (of Lasso for Cowichan Chinook)

In [28]:
#Adapt according to the table found

alpha_selected=0.001

In [29]:
#Look at a particular row (depends on alpha value )
# Make a list of the columns where coefficients are non-zero

selected_row = lassoCoefs.loc["alpha = " + str(alpha_selected)]
lasso_model =selected_row[selected_row != 0].index.tolist()

lasso_model

['rc_Chinook', 'Whale_NRKW', 'av_SeaLevels', 'Av_totalBiomass']

In [30]:
#Print the conclusion
print("According to Lasso analysis, the features", lasso_model, 'have more importance for ', type_name, 'viability.')

According to Lasso analysis, the features ['rc_Chinook', 'Whale_NRKW', 'av_SeaLevels', 'Av_totalBiomass'] have more importance for  Cowichan_Chinook viability.


## Lasso for Big Qualicum Coho

In [31]:
#For different Salmon type adapt the type_number
    ##Different Salmon type might need different alphas

type_number=bqc
type_name=salmonTypes_List[type_number]

In [32]:
#Drop the other Salmon Types from the data

salmon= salmonSelectedFeatures.drop(columns=[salmonTypes_List[i] for i in range(0, 4) if i != type_number])


In [33]:
scaler = StandardScaler()

In [34]:
X=salmon.drop([type_name], axis=1)
scaled_data = scaler.fit_transform(X)
target = salmon[type_name]

In [35]:
#Several Lasso fittings for different alpha values

alphas = [0.000001, 0.00001, 0.0001, 0.00025, 0.0005, 0.00075, 0.001, 0.0025, 0.005, 0.01]

coefs = np.zeros((len(alphas), len(X.columns)))

for i,alpha in enumerate(alphas):
    lasso = Lasso(alpha = alpha, max_iter = 1000000)
    lasso.fit(scaled_data, target)
    
    coefs[i, :] = lasso.coef_
    
lassoCoefs = pd.DataFrame(coefs, 
                index=["alpha = " + str(alpha) for alpha in alphas],
                columns=[col for col in X.columns])

##Display the coefficients
###and display the features that produces non-zero coefficient
print('Lasso Coefficients for', type_name)
display(lassoCoefs)
print("The non-zero features are:")
for alpha in alphas:
    alpha_selected=alpha
    selected_row = lassoCoefs.loc["alpha = " + str(alpha_selected)]
    lasso_model_alpha =selected_row[selected_row != 0].index.tolist()
    print('for', alpha_selected, ': ', lasso_model_alpha, 'with length ', len(lasso_model_alpha))

Lasso Coefficients for Big_Qualicum_Coho


,rc_Chinook,rc_Coho,Seal_sogNt,Whale_NRKW,av_SeaLevels,Av_totalBiomass
alpha = 1e-06,0.000892,0.001502,0.002291,-0.001525,0.002545,0.002429
alpha = 1e-05,0.000898,0.001461,0.002258,-0.001484,0.002503,0.002394
alpha = 0.0001,0.000955,0.001053,0.001930,-0.001071,0.002087,0.002039
alpha = 0.00025,0.001051,0.000374,0.001384,-0.000381,0.001394,0.001448
alpha = 0.0005,0.000957,0.000000,0.000813,-0.000000,0.000653,0.000877
alpha = 0.00075,0.000739,0.000000,0.000411,0.000000,0.000118,0.000511
alpha = 0.001,0.000464,0.000000,0.000222,0.000000,0.000000,0.000198
alpha = 0.0025,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
alpha = 0.005,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
alpha = 0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


The non-zero features are:
for 1e-06 :  ['rc_Chinook', 'rc_Coho', 'Seal_sogNt', 'Whale_NRKW', 'av_SeaLevels', 'Av_totalBiomass'] with length  6
for 1e-05 :  ['rc_Chinook', 'rc_Coho', 'Seal_sogNt', 'Whale_NRKW', 'av_SeaLevels', 'Av_totalBiomass'] with length  6
for 0.0001 :  ['rc_Chinook', 'rc_Coho', 'Seal_sogNt', 'Whale_NRKW', 'av_SeaLevels', 'Av_totalBiomass'] with length  6
for 0.00025 :  ['rc_Chinook', 'rc_Coho', 'Seal_sogNt', 'Whale_NRKW', 'av_SeaLevels', 'Av_totalBiomass'] with length  6
for 0.0005 :  ['rc_Chinook', 'Seal_sogNt', 'av_SeaLevels', 'Av_totalBiomass'] with length  4
for 0.00075 :  ['rc_Chinook', 'Seal_sogNt', 'av_SeaLevels', 'Av_totalBiomass'] with length  4
for 0.001 :  ['rc_Chinook', 'Seal_sogNt', 'Av_totalBiomass'] with length  3
for 0.0025 :  [] with length  0
for 0.005 :  [] with length  0
for 0.01 :  [] with length  0


### Conclusion (of Lasso for Coho)

In [36]:
#Adapt according to the table found

alpha_selected=0.001


In [37]:
#Look at a particular row (given by the selected alpha value )
# Make a list of the columns where coefficients are non-zero

selected_row = lassoCoefs.loc["alpha = " + str(alpha_selected)]
lasso_model =selected_row[selected_row != 0].index.tolist()


In [38]:
#Print the conclusion
print("According to Lasso analysis, the features", lasso_model, 'have more importance for ', type_name, 'viability.')

According to Lasso analysis, the features ['rc_Chinook', 'Seal_sogNt', 'Av_totalBiomass'] have more importance for  Big_Qualicum_Coho viability.


## Ridge for Cowichan Chinook

In [39]:
#For different Salmon type adapt the type_number
    ##Different Salmon type might need different alphas

type_number=cc
type_name=salmonTypes_List[type_number]

In [40]:
#Drop the other Salmon Types from the data

salmon= salmon_cc=salmonSelectedFeatures.drop(columns=[salmonTypes_List[i] for i in range(0, 4) if i != type_number])


In [41]:
scaler = StandardScaler()

In [42]:
X=salmon.drop([type_name], axis=1)
scaled_data = scaler.fit_transform(X)
target = salmon[type_name]

In [43]:
#Several Ridge fittings for different alpha values

alphas =[10**i for i in range(-20, 20, 4)]


coefs = np.zeros((len(alphas), len(X.columns)))

for i,alpha in enumerate(alphas):
    ridge = Ridge(alpha = alpha, max_iter = 1000000)
    ridge.fit(scaled_data, target)
    
    coefs[i, :] = ridge.coef_
    
ridgeCoefs = pd.DataFrame(coefs, 
                index=["alpha = " + str(alpha) for alpha in alphas],
                columns=[col for col in X.columns])

##Display the coefficients
###and display the features that produces max-min coefficient
print('Ridge Coefficients for', type_name)
display(ridgeCoefs)
print("The max-min features are:")
for alpha in alphas:
    alpha_selected=alpha
    selected_row = ridgeCoefs.loc["alpha = " + str(alpha_selected)]
    ridge_model_alpha =selected_row.index.tolist()
    print('for', alpha_selected, ' the max coefficients belongs to ', max(ridge_model_alpha),  'and the min coefficients belongs to ', min(ridge_model_alpha))

Ridge Coefficients for Cowichan_Chinook


,rc_Chinook,rc_Coho,Seal_sogNt,Whale_NRKW,av_SeaLevels,Av_totalBiomass
alpha = 1e-20,2.524664e-03,-7.617630e-04,-3.372833e-06,1.330848e-03,2.238032e-03,2.879024e-03
alpha = 1e-16,2.524664e-03,-7.617630e-04,-3.372833e-06,1.330848e-03,2.238032e-03,2.879024e-03
alpha = 1e-12,2.524664e-03,-7.617630e-04,-3.372833e-06,1.330848e-03,2.238032e-03,2.879024e-03
alpha = 1e-08,2.524664e-03,-7.617630e-04,-3.372832e-06,1.330848e-03,2.238032e-03,2.879024e-03
alpha = 0.0001,2.524614e-03,-7.617332e-04,-3.365775e-06,1.330858e-03,2.238013e-03,2.878996e-03
alpha = 1,2.128009e-03,-5.414382e-04,3.885144e-05,1.401464e-03,2.051506e-03,2.623058e-03
alpha = 10000,2.863802e-06,9.056044e-07,3.185436e-07,4.634414e-06,2.805185e-06,3.350852e-06
alpha = 100000000,2.871035e-10,9.108989e-11,3.197834e-11,4.645488e-10,2.809607e-10,3.354651e-10
alpha = 1000000000000,2.871035e-14,9.108994e-15,3.197835e-15,4.645489e-14,2.809608e-14,3.354652e-14
alpha = 10000000000000000,2.871035e-18,9.108994e-19,3.197835e-19,4.645489e-18,2.809608e-18,3.354652e-18


The max-min features are:
for 1e-20  the max coefficients belongs to  rc_Coho and the min coefficients belongs to  Av_totalBiomass
for 1e-16  the max coefficients belongs to  rc_Coho and the min coefficients belongs to  Av_totalBiomass
for 1e-12  the max coefficients belongs to  rc_Coho and the min coefficients belongs to  Av_totalBiomass
for 1e-08  the max coefficients belongs to  rc_Coho and the min coefficients belongs to  Av_totalBiomass
for 0.0001  the max coefficients belongs to  rc_Coho and the min coefficients belongs to  Av_totalBiomass
for 1  the max coefficients belongs to  rc_Coho and the min coefficients belongs to  Av_totalBiomass
for 10000  the max coefficients belongs to  rc_Coho and the min coefficients belongs to  Av_totalBiomass
for 100000000  the max coefficients belongs to  rc_Coho and the min coefficients belongs to  Av_totalBiomass
for 1000000000000  the max coefficients belongs to  rc_Coho and the min coefficients belongs to  Av_totalBiomass
for 1000000000000000